In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import sys
sys.path.append(r"src")
from cleaning import*

In [2]:
# We don't want any pandas warnings because we're reckless :^)
pd.options.mode.chained_assignment = None

In [3]:
# I get the initial dataframe from Kaggle, it's a list of the top 200 rated players in chess: 
# their name, rating(ELO), birth year and federation
df = pd.read_csv('Input/Fide_Rankings.csv')

In [4]:
# Cleaning spaces in column names
df = df.rename(columns={'name ':'name', 'federation ' :'federation', 'ELO ':'ELO'})

In [5]:
# Cleaning spaces of all the names in the name column
df["name"] = df["name"].apply(strip)

In [6]:
# Calculating current age (actually age at the end of 2022) and writing it in a new column "age"
df["age"] = df["birth_year"].apply(age)

In [7]:
# Some chinese/vietnamese names were written "Surname"+"Name" and this functions rewrites them as "Name"+"Surname"
# This is done so I can merge this dataframe with another one that uses "Name"+"Surname" for chinese names
china(df)

In [8]:
#I scrap top players ratings in faster time controls and write them in a new dataframe
fide_df = scrapping()

In [9]:
#I manually change some of the names in the first dataframe so they match the names in the second dataframe (so I can join them)
df = replace_names(df)

In [10]:
#I merge the 2 dataframes by name
df_merged = df.merge(fide_df, how = 'left', on = 'name')

In [11]:
#I drop the 3 entries where I don't have data for blitz rating
df_merged = df_merged.drop(df_merged[df_merged.blitz == 0].index)

In [12]:
#The resulting dataframe is then exported
df_merged.to_csv("Output/enriched_data.csv")